# YOLOv3 with Pytorch on Colab

In [0]:
#Installing PyTorch 
!pip install torch==0.4
!wget https://raw.githubusercontent.com/AvivSham/YOLO_V3_from_scratch_colab/master/util.py
!wget https://raw.githubusercontent.com/AvivSham/YOLO_V3_from_scratch_colab/master/darknet.py
!wget https://raw.githubusercontent.com/AvivSham/YOLO_V3_from_scratch_colab/master/pallete
  

     |████████████████████████████████| 484.0MB 65kB/s 
ERROR: torchvision 0.4.1+cu100 has requirement torch==1.3.0, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.58 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.3.0+cu100
    Uninstalling torch-1.3.0+cu100:
      Successfully uninstalled torch-1.3.0+cu100
--2019-10-20 11:38:07--  https://raw.githubusercontent.com/AvivSham/YOLO_V3_from_scratch_colab/master/util.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7432 (7.3K) [text/plain]
Saving to: ‘util.py’

util.py             100%[===================>]   7.26K  --.-KB/s    in 0s      

2019-10-20 11:38:07 (149 MB/s) - ‘util.py’ saved [7432/7432]

--2019-10-20 11:38:11

In [0]:
#Import dependencies
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from darknet import *
import pickle as pkl
import pandas as pd
import random
import csv

In [0]:
#Get weights and graph file for the model

#Get weights file for the model
!wget https://pjreddie.com/media/files/yolov3.weights
# Get YOLO graph file
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg


--2019-10-20 11:38:31--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  3.77MB/s    in 44s     

2019-10-20 11:39:15 (5.41 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

--2019-10-20 11:39:17--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2019

In [0]:
#Setting parameters

# images --> Input image / directory of images.
# reso   --> Input image resolution
# detect_image --> flag to decide to detect video or single photo

images = os.getcwd() + '/Images'
images_det = os.getcwd() + '/Images_Detection'
#print(images, images_det)
batch_size = 1
confidence = 0.4
nms_thesh = 0.5
reso = 416
weights_file = "yolov3.weights"
config_file = "yolov3.cfg"
start = 0
detect_image = False
CUDA = torch.cuda.is_available()

In [0]:
# Making directory for COCO names file
!mkdir data
os.chdir('./data')

In [0]:
#COCO names file
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names

--2019-10-20 11:39:37--  https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names’

coco.names          100%[===================>]     625  --.-KB/s    in 0s      

2019-10-20 11:39:37 (125 MB/s) - ‘coco.names’ saved [625/625]



In [0]:
# COCO dataset has 80 different classes
num_classes = 80
classes = load_classes("coco.names")

In [0]:
%cd ..

/content


In [0]:
#Install pytube to download videos from youtube
!pip install pytube

In [0]:
#Download youtube video
from pytube import YouTube
YouTube('https://www.youtube.com/watch?v=0EDc-DO287I').streams.first().download()

In [0]:
#If youtube video can't be downloaded
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving WIN_20191020_21_13_39_Pro.mp4 to WIN_20191020_21_13_39_Pro.mp4
User uploaded file "WIN_20191020_21_13_39_Pro.mp4" with length 10036286 bytes


In [0]:
def load_classes(namefile):
  fp = open(namefile, 'r')
  names = fp.read().split("\n")[:-1] #discard the last
  return names  

In [0]:
def distance_to_camera(knownWidth,perwidth,label):
    # compute and return the distance from the maker to the camera
    global objects
    global mydict
    if(label in objects):
        print("Im here")
        focallength=0.75*perwidth
        mydict[label]=focallength
        objects.remove(label)
        mydict['cell phone']=99.5    
    print("Object: "+str(label))
    print("perwidth: "+str(perwidth))  
    print("label focallength: "+str(mydict[label]))                 
    return (knownWidth * mydict[label]) / (perwidth)

def write_video_box(x, results):
  c1 = tuple(x[1:3].int())
  c2 = tuple(x[3:5].int())
  c1=(int(c1[0]),int(c1[1]))  #Tuple 1
  c2=(int(c2[0]),int(c2[1]))  #Tuple 2
  img = results
  cls = int(x[-1])
  width=int(x[3])-int(x[1])
  height=int(x[2])-int(x[4])
  color = random.choice(colors)
  label = "{0}".format(classes[cls])
  cv2.rectangle(img, c1, c2,color,10)
  inches = distance_to_camera(24.0,width,label)/12
  print(inches)
  t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
  c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
  cv2.rectangle(img, c2, c2,color,-1)
  cv2.putText(img, label+" "+str(round(inches,2))+"ft", (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 3, [255,255,255], 3);
  return img

In [0]:
# Set up the network
objects=['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'fire hydrant','traffic light', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'potted', 'plant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwaveoven', 'toasters', 'ink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy', 'bear', 'hair drier', 'toothbrush']
mydict={}
print("Loading network...")
model = Darknet(config_file)
model.load_weights(weights_file)
print("Network successfully loaded")


model.net_info["height"] = reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0
assert inp_dim > 32

# if GPU is available allocate the model
if CUDA:
  model.cuda()
  
# Set the model in evaluation mode
model.eval()
if detect_image==False:
  #Detection phase
  #Use cv2.VideoCapture('your video file name or path')
  cap = cv2.VideoCapture('WIN_20191020_21_13_39_Pro.mp4') #cap = cv.VideoCapture(0)
  
  fps = cap.get(cv2.CAP_PROP_FPS)
  sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
       int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
  fourcc = cv2.VideoWriter_fourcc(*'DIVX')
  vout = cv2.VideoWriter()
  vout.open('./video_output1.avi',fourcc,fps,sz,True)

  
  assert cap.isOpened(), 'Cannot capture source'

  frames = 0 
  start = time.time()

  while cap.isOpened():
      ret, frame = cap.read()

      if ret:   
          img = prep_image(frame, inp_dim)
          im_dim = frame.shape[1], frame.shape[0]
          im_dim = torch.FloatTensor(im_dim).repeat(1,2)   

          if CUDA:
              im_dim = im_dim.cuda()
              img = img.cuda()

          with torch.no_grad():
              output = model(Variable(img, volatile = True), CUDA)
          output = write_results(output, confidence, num_classes, nms_conf = nms_thesh)


          if type(output) == int:
              frames += 1
              key = cv2.waitKey(1)
              if key & 0xFF == ord('q'):
                  break
              continue

          im_dim = im_dim.repeat(output.size(0), 1)
          scaling_factor = torch.min(416/im_dim,1)[0].view(-1,1)

          output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
          output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2

          output[:,1:5] /= scaling_factor

          for i in range(output.shape[0]):
              output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
              output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])


          classes = load_classes('data/coco.names')
          colors = pkl.load(open("pallete", "rb"))

          list(map(lambda x: write_video_box(x, frame), output))
          
          vout.write(frame)
          key = cv2.waitKey(1)
          if key & 0xFF == ord('q'):
              FPS = frames // (time.time() - start)
              break
          frames += 1
          
      else:
          FPS = frames // (time.time() - start)
          break

Loading network...
Network successfully loaded


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Im here
Object: person
perwidth: 678
label focallength: 508.5
1.5
Object: person
perwidth: 682
label focallength: 508.5
1.4912023460410557
Object: person
perwidth: 684
label focallength: 508.5
1.4868421052631577
Object: person
perwidth: 682
label focallength: 508.5
1.4912023460410557
Object: person
perwidth: 693
label focallength: 508.5
1.4675324675324675
Object: person
perwidth: 688
label focallength: 508.5
1.4781976744186045
Object: person
perwidth: 694
label focallength: 508.5
1.4654178674351586
Object: person
perwidth: 684
label focallength: 508.5
1.4868421052631577
Object: person
perwidth: 678
label focallength: 508.5
1.5
Object: person
perwidth: 679
label focallength: 508.5
1.4977908689248896
Object: person
perwidth: 687
label focallength: 508.5
1.4803493449781657
Object: person
perwidth: 682
label focallength: 508.5
1.4912023460410557
Object: person
perwidth: 680
label focallength: 508.5
1.4955882352941174
Object: person
perwidth: 681
label focallength: 508.5
1.4933920704845816


In [0]:
#Mounting Drive  (Optional)
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
files.download('video_output1.avi') 

In [0]:
!rm video_output.avi

rm: cannot remove 'video_output.avi': No such file or directory


In [0]:
#Copying result video to drive (Optional)
!cp /content/video_output1.avi /content/drive/'My Drive'/

cp: cannot stat '/content/video_output1.avi': No such file or directory
